In [1]:
import requests
import pandas as pd
from zipfile import ZipFile
import shutil
from shapely.ops import unary_union
from tqdm.notebook import tqdm

import os

# NYC Bikes #

In [4]:
city = 'NYC'
transp = 'Bike'
for i in tqdm(range (1,13)):
    for year in range (2018,2020):
        filename ="JC-" + str(year)+ "%.2d" % i +'-citibike-tripdata.csv.zip'
        url = "https://s3.amazonaws.com/tripdata/" + filename
        req = requests.get(url)
        url_content = req.content
        directory = "./data/"+transp+city+"/"
        if not os.path.exists(directory):
            os.makedirs(directory)

        csv_file = open(directory + filename, 'wb')
        csv_file.write(url_content)
        csv_file.close()

  0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
for filename in os.listdir(directory):
    if filename != '.DS_Store':
        print(directory + filename)
        with ZipFile(directory + filename, 'r') as zipObject:
            listOfFileNames = zipObject.namelist()
            for fileName in listOfFileNames:
                if fileName.endswith('.csv'):
                    # Extract a single file from zip
                    zipObject.extract(fileName, directory)
            print('All the csv files are extracted')


./data/BikeNYC/JC-201911-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201908-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201809-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201901-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201808-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201804-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201810-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201803-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201805-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201806-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201907-citibike-tripdata.csv.zip
All the csv files are extracted
./data/BikeNYC/JC-201909-citibike-tripdata.csv.zip
All the csv files are extracted
./da

In [6]:
for filename in os.listdir(directory):
    if filename.endswith(".zip"):
            os.remove(os.path.join(directory, filename))
    if filename == '__MACOSX':
        shutil.rmtree(os.path.join(directory, filename))
    if filename == '.DS_Store':
        os.remove(os.path.join(directory, filename))

In [7]:
#open the first file to create the dataframe
df = pd.read_csv(directory + os.listdir(directory)[0])
df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,284,2018-09-01 01:48:22.0320,2018-09-01 01:53:06.9130,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,26303,Subscriber,1984,1
1,370,2018-09-01 01:48:22.5010,2018-09-01 01:54:32.9270,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,29667,Subscriber,1984,1
2,1181,2018-09-01 09:37:04.3070,2018-09-01 09:56:45.3280,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.055701,26307,Customer,1994,1
3,383,2018-09-01 09:44:46.0030,2018-09-01 09:51:09.9230,3183,Exchange Place,40.716247,-74.033459,3213,Van Vorst Park,40.718489,-74.047727,29261,Subscriber,1987,1
4,377,2018-09-01 09:44:46.1850,2018-09-01 09:51:03.6330,3183,Exchange Place,40.716247,-74.033459,3213,Van Vorst Park,40.718489,-74.047727,33665,Subscriber,1985,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39017,985,2018-09-29 09:02:31.3090,2018-09-29 09:18:56.6570,3694,Jackson Square,40.711130,-74.078900,3211,Newark Ave,40.721525,-74.046305,29492,Subscriber,1991,2
39018,1595,2018-09-30 09:36:44.8650,2018-09-30 10:03:19.9750,3694,Jackson Square,40.711130,-74.078900,3183,Exchange Place,40.716247,-74.033459,29252,Subscriber,1987,1
39019,1996,2018-09-30 11:15:46.8100,2018-09-30 11:49:03.1200,3694,Jackson Square,40.711130,-74.078900,3186,Grove St PATH,40.719586,-74.043117,29122,Customer,1987,2
39020,1803,2018-09-30 11:16:37.0760,2018-09-30 11:46:40.6460,3694,Jackson Square,40.711130,-74.078900,3273,Manila & 1st,40.721651,-74.042884,29251,Customer,1974,1
